In [35]:
%matplotlib widget

In [2]:
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
from fredapi import Fred as FredApi

fred = FredApi(api_key='0dfcbb403eea7fee006f01e935be9216')

cpiausl = fred.get_series_latest_release('CPIAUCSL')
brent_daily = fred.get_series_latest_release('DCOILBRENTEU')

In [3]:
# verify what the latest number available is
cpiausl.tail()

2021-07-01    272.265
2021-08-01    273.012
2021-09-01    274.138
2021-10-01    276.724
2021-11-01    278.880
dtype: float64

In [10]:
# the brent series is daily - so i resample monthly and take an average
brent = brent_daily.resample('M').mean()

In [11]:
# see where the Brent series stops first (only month now is relevant)
brent.tail()

2021-09-30    74.488636
2021-10-31    83.538095
2021-11-30    81.050000
2021-12-31    74.170476
2022-01-31    78.250000
Freq: M, dtype: float64

In [13]:
# it's useful to make sure that Python understands the frequency being "monthly" even though data
# don't necessarily show this. Resampling is a way around it and selecting "last" as the value of the
# resampling (there is only 1 value)
# compare the indices of the prior version of "brent" to this version of "brent" (note reassignment)
brent = brent.resample('M').last()
brent_yoy = brent.pct_change(periods=12) * 100.
brent.tail()

2021-09-30    74.488636
2021-10-31    83.538095
2021-11-30    81.050000
2021-12-31    74.170476
2022-01-31    78.250000
Freq: M, dtype: float64

In [7]:
# now extend the brent timeseries with a constant one
new_index = pd.date_range(start=brent.index[-1], freq='M', periods=12)
brent_projection = pd.Series(data=brent.iloc[-1], index=new_index)
# concatenate the two brent timeseries one to the other
brent_extended = pd.concat([brent, brent_projection.iloc[1:]])
brent_extended.tail(24)

2021-01-31    54.774000
2021-02-28    62.276500
2021-03-31    65.410000
2021-04-30    64.806500
2021-05-31    68.534211
2021-06-30    73.164091
2021-07-31    75.165909
2021-08-31    70.747143
2021-09-30    74.488636
2021-10-31    83.538095
2021-11-30    81.050000
2021-12-31    74.170476
2022-01-31    78.250000
2022-02-28    78.250000
2022-03-31    78.250000
2022-04-30    78.250000
2022-05-31    78.250000
2022-06-30    78.250000
2022-07-31    78.250000
2022-08-31    78.250000
2022-09-30    78.250000
2022-10-31    78.250000
2022-11-30    78.250000
2022-12-31    78.250000
Freq: M, dtype: float64

In [8]:
# make a yoy variation as a % change for the CPI
# note the change in index as well here... same as above
cpiausl = cpiausl.resample('M').last()
cpiausl_yoy = cpiausl.pct_change(periods=12) * 100.

# make a yoy variation as a % change for the extended brent projection
# and then check the numbers vs. what you have above to see if it makes
# sense (good practice!)
brent_extended_yoy = brent_extended.pct_change(periods=12) * 100.
brent_extended_yoy.tail(12)

2022-01-31    42.859751
2022-02-28    25.649322
2022-03-31    19.630026
2022-04-30    20.744061
2022-05-31    14.176554
2022-06-30     6.951373
2022-07-31     4.103045
2022-08-31    10.605173
2022-09-30     5.049580
2022-10-31    -6.330160
2022-11-30    -3.454658
2022-12-31     5.500199
Freq: M, dtype: float64

In [37]:
# plot and set labels of the figure
START_YEAR = '2000'
fig, ax = plt.subplots()
cpiausl_yoy.loc[START_YEAR:].plot(ax=ax, linewidth=5);
ax2 = brent_extended_yoy.loc[START_YEAR:].plot(secondary_y=True, ax=ax);
ax.grid(True);
ax.set_ylabel('CPI YoY (%)');
ax2.set_ylabel('Brent YoY (%)');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [42]:
# now let's try to cross-correlate these 2 indicators to see the lags at which there is some correlation 
# and how big that is. To do so, we only look at the real data available, not the extended brent series.
import statsmodels.tsa.stattools as smtsast
common_dates = brent_yoy.loc[~brent_yoy.isna()].index.intersection(cpiausl_yoy.index)
cc = smtsast.ccf(brent_yoy.loc[common_dates], cpiausl_yoy.loc[common_dates])
cross_correl_function = pd.Series(cc[:60])
fig, ax = plt.subplots()
cross_correl_function.plot(ax=ax, linewidth=1, style='o--');
ax.grid(True);
ax.set_xlabel('lag (months) CPI vs. Brent');
ax.set_ylabel('cross correlation function');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …